## Ripple Counter from Toggle Flip-Flops

In this example we create a ripple counter from toggle flip-flops. 
We also show how to define new `Magma` `Circuits`
and introduce *generators*.

In [1]:
import magma as m
m.set_mantle_target("ice40")

In the last example, we defined a function that created a
toggle flip-flop (TFF) from a DFF and an XOR gate.
Let's convert the TFF to a `Circuit`. 
In `Magma` a `Circuit` is equivalent to a verilog *module*.
Circuits can be instanced and then wired to other circuits.

`m.ClockInterface()` appends `Magma`'s standard clock interface ports to the interface. When no parameters are specified, this just adds the port `CLK` with type `In(Clock)`.

In [2]:
from mantle import DFF

class TFF(m.Circuit):
    IO = ["O", m.Out(m.Bit)] + m.ClockInterface()
    @classmethod
    def definition(io):
        ff = DFF()
        m.wire( ff(~ff.O), io.O )
        m.wireclock( io, ff )

import lattice ice40
import lattice mantle40


Let's inspect the interface to see the result of appending `m.ClockInterface()`.

In [3]:
print(TFF)

TFF(O: Out(Bit), CLK: In(Clock))


Now we'll define a *generator* for our RippleCounter that accepts a single argument `n` that corresponds to the width. A *generator* in `Magma` simply refers to a native Python function that defines and returns `Magma` Circuit. We use the naming convention `Define<CircuitName>` to distinguish `Magma` generators from other Python functions.

In [4]:
def DefineRippleCounter(n):
    class RippleCounter(m.Circuit):
        name = 'Ripple' + str(n)
        IO = ["O", m.Out(m.Bits(n))] + m.ClockInterface()
        @classmethod
        def definition(io):
            tffs = [TFF() for i in range(n)]
            O = io.CLK
            for i in range(n):
                m.wire(O, tffs[i].CLK)
                O = tffs[i].O
                m.wire(O, io.O[i])
    return RippleCounter

Now we can generate a 30-bit `RippleCounter` by calling our function.

In [5]:
N = 30
Ripple = DefineRippleCounter(N)

We'll test our circuit by wiring the top 8 bits of the ripple counter to `J3` header on the IceStick.

In [6]:
from loam.boards.icestick import IceStick

icestick = IceStick()
icestick.Clock.on()
icestick.J1[0].rename('J1').input().on()  # rename so J1 is not an array
for i in range(8):
    icestick.J3[i].output().on()

main = icestick.DefineMain()
ripple = Ripple()
m.wire( ripple.O[N-8:N], main.J3 )
m.EndDefine()

## Compile and Build

In [7]:
m.compile("build/ripple", main)

compiling TFF
compiling Ripple30
compiling main


In [8]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif ripple.blif' ripple.v
arachne-pnr -q -d 1k -o ripple.txt -p ripple.pcf ripple.blif 
icepack ripple.txt ripple.bin
iceprog ripple.bin

init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x35 0x00 0x35 0x06 0x06 0x15 0x43 0xB6
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.


We verify our circuit using a logic analyzer set to sample at 1 MS/s over 45 seconds

![](images/ripple.png)